### Task

In [7]:
# from dotenv import load_dotenv
# import os

# load_dotenv()
# API_KEY = os.getenv("OPENWEATHER_API_KEY")
# API_KEY

None


In [1]:
#### TEST
import requests
import pandas as pd
from datetime import datetime
import time
import pytz
from datetime import timedelta

import nest_asyncio
import asyncio
import aiohttp
from prefect import flow, task # Prefect flow and task decorators
from math import ceil

from dotenv import load_dotenv
import os

nest_asyncio.apply()

load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")
ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")

@task
async def fetch_pollution_data(df_sample, batch_size=100):  #sample
    # API_KEY = "13fd31c548dbeae77a5c0d773a99df79"
    # API_KEY = os.getenv("OPENWEATHER_API_KEY")
    # WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"
    
    async def fetch_row(session, row):
        # await asyncio.sleep(1)  #พัก 1 วิแบบไม่บล็อก loop
        lat = row['lat']
        lon = row['lon']
        province = row['provinceEN']
        district = row['amphoeEN']
        try:
            params = {
                "lat" : lat,
                "lon" : lon,
                "appid": API_KEY,
                "units": "metric"
            }
            async with session.get(POLLUTION_ENDPOINT, params=params) as response:
                # if response.status != 200:
                #     return {
                #         'province': province,
                #         'amphoe': amphoe,
                #         'lat': lat,
                #         'lon': lon,
                #         'error': f"HTTP {response.status}"
                #     }
                data = await response.json()
                dt = datetime.utcnow()
                thai_tz = pytz.timezone('Asia/Bangkok')
                localtime = dt.astimezone(thai_tz)
                components = data['list'][0]['components']
                pollution_dict = {
                    'timestamp': dt,
                    'year': dt.year,
                    'month': dt.month,
                    'day': dt.day,
                    'hour': dt.hour,
                    'minute': dt.minute,
                    'localtime': localtime,
                    'province' : province,
                    'district' : district,
                    'lat' : data['coord']['lat'],
                    'lon' : data['coord']['lon'],
                    # 'location': data['name'],
                    'main.aqi' : data['list'][0]['main']['aqi'],
                    'components_co' : components['co'],
                    'components_no' : components['no'],
                    'components_no2' : components['no2'],
                    'components_o3' : components['o3'],
                    'components_so2' : components['so2'],
                    'components_pm2_5' : components['pm2_5'],
                    'components_pm10' : components['pm10'],
                    'components_nh3' : components['nh3']
                    }
                return pollution_dict

        # # Check Error
        # except Exception as e:
        #     return {
        #         'province': row.get('province', 'unknown'),
        #         'amphoe': row.get('amphoe', 'unknown'),
        #         'lat': row.get('lat', None),
        #         'lon': row.get('lon', None),
        #         'error': str(e)
        # }
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return None
        except KeyError as e:
            print(f"Error processing data: Missing key {e}")
            return None
        except Exception as e:
            print(f"Error: {e} at {province} - {district}")
            return None



    pollution_results = []
    total_batches = ceil(len(df_sample) / batch_size)  #sample
    async with aiohttp.ClientSession() as session:
        for i in range(total_batches):
            batch = df_sample.iloc[i*batch_size:(i+1)*batch_size]
            tasks = [fetch_row(session, row) for _, row in batch.iterrows()]
            batch_results = await asyncio.gather(*tasks)
            pollution_results.extend(batch_results)

            print(f"✅ เสร็จ batch {i+1}/{total_batches}")
            if i < total_batches - 1:
                await asyncio.sleep(65)  # ✅ รอให้ผ่าน rate limit

    return pollution_results


def clean_data(df):
    df = pd.DataFrame(df)
    # if 'error' in df.columns:
    #     # df = df[df['error'].isna()]  # กรอง row ที่มี error ออก
    #     df = df.drop(columns=['error'], errors='ignore')  # ลบคอลัมน์ error
    df['province'] = df['province'].astype("string")
    df['district'] = df['district'].astype("string")

    return df


def save_to_lakefs(df):
    # ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
    # SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
    # lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")
    # ACCESS_KEY = "access_key"
    # SECRET_KEY = "secret_key"
    # lakefs_endpoint = "http://lakefs-dev:8000/"

    repo = "pollution-data"
    branch = "main"
    path = "pollution.parquet"
    
    lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

    storage_options = {
        "key": ACCESS_KEY,
        "secret": SECRET_KEY,
        "client_kwargs": {"endpoint_url": lakefs_endpoint}
    }

    df.to_parquet(
        lakefs_s3_path,
        storage_options=storage_options,
        partition_cols=["year", "month", "day", "hour"],
    )



### FLOW

In [2]:
import time
import os
import pandas as pd

@flow(name="pollution-flow", log_prints=True)
async def pollution_flow():
    start_time = time.perf_counter()  # ⏱ เริ่มจับเวลา

    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work
    coord_path = os.path.join(BASE_DIR, "save", "amphoe_coord.csv")
    coord_df = pd.read_csv(coord_path)
    df_sample = coord_df.sample(10)
    pollution_results = await fetch_pollution_data(df_sample)
    # pollution_data = pd.DataFrame(pollution_results)

    # #Check Error
    # df_raw = pd.DataFrame(pollution_results)
    # if 'error' in df_raw.columns:
    #     error_df = df_raw[df_raw['error'].notna()]
    #     print("\n❗ Error Rows:")
    #     print(error_df[['province', 'amphoe', 'error']].head())
    pollution_data = clean_data(pollution_results)
    
    
    end_time = time.perf_counter()  # ⏱ จับเวลาอีกครั้ง
    print(f"\n✅ ดึงข้อมูลเสร็จทั้งหมด ใช้เวลา {end_time - start_time:.2f} วินาที")
    save_to_lakefs(pollution_data) 
    print("save to Lakefs Success")
    print(pollution_data.info())
    print(pollution_data.head())
    # return pollution_data
    

In [9]:
# #Check path + Sample
# BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work
# coord_path = os.path.join(BASE_DIR, "save", "amphoe_coord.csv")
# coord_df = pd.read_csv(coord_path)
# df_sample = coord_df.sample(10)
# df_sample

,province_id,province,provinceEN,amphoe_id,amphoe,amphoeEN,lat,lon
77,46,กาฬสินธุ์,Kalasin,4614,ห้วยผึ้ง,Huai Phueng,16.652100,103.891200
262,30,นครราชสีมา,Nakhon Ratchasima,3004,คง,Khong,15.422308,102.419010
626,90,สงขลา,Songkhla,9005,เทพา,Thepha,6.797260,100.910250
183,86,ชุมพร,Chumphon,8606,พะโต๊ะ,Phato,9.791667,98.776667
456,66,พิจิตร,Phichit,6605,บางมูลนาก,Bang Mun Nak,16.021400,100.417200
57,10,กรุงเทพมหานคร,Bangkok,1012,ยานนาวา,Yan Nawa,13.694300,100.539400
518,21,ระยอง,Rayong,2108,นิคมพัฒนา,Nikhom Phatthana,14.909170,101.026730
209,63,ตาก,Tak,6303,สามเงา,Sam Ngao,17.243310,99.022560
329,55,น่าน,Nan,5505,ปัว,Pua,19.175000,100.916000
59,71,กาญจนบุรี,Kanchanaburi,7101,เมืองกาญจนบุรี,Mueang Kanchanaburi,14.069870,99.327690


### TEST

In [3]:
# test-run pollution flow
import nest_asyncio
nest_asyncio.apply()

await pollution_flow()


11:30:06.548 | INFO    | prefect.engine - Created flow run 'capable-galago' for flow 'pollution-flow'

11:30:06.557 | INFO    | prefect.engine - View at http://prefect-server:4200/runs/flow-run/ac2547bc-19ae-49ff-be3e-f101a87cdc6b

11:30:06.847 | INFO    | Task run 'fetch_pollution_data-0' - Created task run 'fetch_pollution_data-0' for task 'fetch_pollution_data'

11:30:07.254 | INFO    | Task run 'fetch_pollution_data-0' - ✅ เสร็จ batch 1/1

11:30:07.281 | INFO    | Task run 'fetch_pollution_data-0' - Finished in state Completed()

...

11:30:07.769 | INFO    | Flow run 'capable-galago' - save to Lakefs Success

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype                       
---  ------            --------------  -----                       
 0   timestamp         10 non-null     datetime64[ns]              
 1   year              10 non-null     int64                       
 2   month             10 non-null     int64                       
 3   day               10 non-null     int64                       
 4   hour              10 non-null     int64                       
 5   minute            10 non-null     int64                       
 6   localtime         10 non-null     datetime64[ns, Asia/Bangkok]
 7   province          10 non-null     string                      
 8   district          10 non-null     string                      
 9   lat               10 non-null     float64                     
 10  lon               10 non-null     float64                     
 11  main.aqi 

11:30:07.775 | INFO    | Flow run 'capable-galago' - None

11:30:07.782 | INFO    | Flow run 'capable-galago' -                    timestamp  year  month  day  hour  minute  \
0 2025-05-11 11:30:07.254435  2025      5   11    11      30   
1 2025-05-11 11:30:07.253874  2025      5   11    11      30   
2 2025-05-11 11:30:07.254321  2025      5   11    11      30   
3 2025-05-11 11:30:07.253944  2025      5   11    11      30   
4 2025-05-11 11:30:07.253634  2025      5   11    11      30   

                         localtime      province        district      lat  \
0 2025-05-11 18:30:07.254435+07:00      Yasothon        Kut Chum  16.0317   
1 2025-05-11 18:30:07.253874+07:00  Kanchanaburi  Thong Pha Phum  14.8328   
2 2025-05-11 18:30:07.254321+07:00        Roi Et         Nong Hi  15.5798   
3 2025-05-11 18:30:07.253944+07:00    Chiang Mai     Wiang Haeng  19.6004   
4 2025-05-11 18:30:07.253634+07:00           Tak   Tha Song Yang  17.2267   

        lon  main.aqi  components_co  components_no  components_no2  \
0  104.3400         2         587.27           0.02           11.59   
1   98.6923         1         157.87           0.00            0.23   
2  104.0113         2         449.83           0.02           10.88   
3   98.6161         1         290.52           0.01            2.98   
4   98.2250         1         234.70           0.00            1.30   

   components_o3  components_so2  components_pm2_5  components_pm10  \
0          23.12            1.41             23.72            24.84   
1          37.81            0.05              1.33             1.80   
2          22.59            1.56             16.46            17.35   
3          43.55            1.13              7.35             7.87   
4          54.29            0.31              6.40             7.11   

   components_nh3  
0            5.42  
1            0.40  
2           10.32  
3            1.25  
4            1.21

11:30:07.813 | INFO    | Flow run 'capable-galago' - Finished in state Completed()

In [ ]:
def clean_data(results):
    # pollution_result = pd.DataFrame(results)
    pollution_result = pollution_result[pollution_result['error'].isna()]  # กรอง error ออก
    return pollution_result.drop(columns=['error'], errors='ignore')

# แยก clean กับ error ออก
clean_results = [r for r in results if 'error' not in r]
error_results = [r for r in results if 'error' in r]

# สร้าง DataFrame
clean_df = pd.DataFrame(clean_results)
error_df = pd.DataFrame(error_results)

# Save ไฟล์ clean ขึ้น LakeFS (หรือ local เตรียมอัป)
clean_df.to_parquet("pollution_data.parquet", index=False)

# เก็บ error ไว้ดูภายหลัง (optional)
if not error_df.empty:
    error_df.to_csv("pollution_errors.csv", index=False)



### LakeFS

In [ ]:
 # lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"
    
# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"
    
    # lakeFS repository, branch, and file path
repo = "pollution-data"
branch = "main"
path = "pollution.parquet"
    
    # Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"
    
    # Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
    }
pollution_df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour'],
        
)

In [ ]:
from prefect import flow, task

@task
async def fetch_pollution_data(...):
    ...

def clean_data(results):
    df = pd.DataFrame(results)
    df = df[df['error'].isna()]  # กรอง error ออก
    return df.drop(columns=['error'], errors='ignore')

def save_to_lakefs(df, path):
    df.to_parquet(path, index=False)

@flow
async def pollution_pipeline():
    raw_results = await fetch_pollution_data(...)
    clean_df = clean_data(raw_results)
    save_to_lakefs(clean_df, "s3://lakefs/bucket/pollution.parquet")
